In [1]:
import pandas as pd
import numpy as np
import polars as pl
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import gzip
import shutil
import pathlib
import os
import sqlalchemy
import sqlite3
import spacy
import re
import tqdm
from tqdm.notebook import tqdm, trange
import ipywidgets as widgets
from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()

%matplotlib inline
alt.data_transformers.disable_max_rows()

nlp = spacy.load("en_core_web_sm")

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
# progress_bar = IntProgress(min=0,max=100, value=0)
#
# label = HTML(value='Progress: 0%')
#
# display(VBox([label, progress_bar]))

# for i in range(1010):
#     progress_bar.value = i
#     label.value = f'Progress: {i}%'
#     time.sleep(0.1)

In [4]:
def convert_path(path):
    converted_path = pathlib.PureWindowsPath(path).as_posix()
    return converted_path

def print_files_in_directory(directory_path):
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file():
                print(entry.name)

def calculate_sentiment_score(tokens):
    sentiment_scores = [token.sentiment for token in nlp(" ".join(tokens))]
    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)
    return avg_sentiment

def tokenize_and_lemmatize(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return tokens

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]','',text)
    return text

#conn = sqlite3.connect('AirBnB_Data.db')
conn = sqlite3.connect(r"C:\Users\asl4a\AirBnB_Data.db")
cursor = conn.cursor()

In [5]:
gz_calendar_path = convert_path("F:\Data Science\Datasets\AirBnB Data\Calendars gz")
gz_listings_path = convert_path("F:\Data Science\Datasets\AirBnB Data\Listings gz")
gz_reviews_path = convert_path("F:\Data Science\Datasets\AirBnB Data\Reviews gz")

calendars_dest_path = convert_path("F:\Data Science\Datasets\AirBnB Data\Calendars")
listings_dest_path = convert_path("F:\Data Science\Datasets\AirBnB Data\Listings")
reviews_dest_path = convert_path("F:\Data Science\Datasets\AirBnB Data\Reviews")

In [9]:
# df_reviews = pd.read_sql("""
# SELECT *
# FROM reviews;""",con=conn)


test_query = pd.read_sql_query("""
SELECT *
FROM reviews;""",con=conn)

In [10]:
test_query.head()

,index,temp_index,listing_id,id,date,reviewer_id,reviewer_name,comments
0,0,0,6422,1927,2009-04-30,14100,Melissa,I can't say enough about how wonderful it was ...
1,1,1,6422,3867,2009-06-11,17413,Raquel,Michelle and Collier's home is wonderful! They...
2,2,2,6422,4159,2009-06-17,20253,Ulrike,I spent one night at Michele's home and felt j...
3,3,3,6422,5724,2009-07-18,22544,Phil,Michele and Collier are two of the loveliest p...
4,4,4,6422,11891,2009-09-29,33409,Claire,We had the most lovely time staying with Miche...


In [6]:
df_reviews = pd.read_csv(r"C:\Users\asl4a\df_reviews.csv")
df_reviews['comments'] = df_reviews['comments'].astype(str)
df_reviews = df_reviews.rename(columns={'Unnamed: 0': 'temp_index'})
df_reviews['cleaned_text'] = df_reviews['comments'].apply(clean_text)

In [7]:
df_reviews.head()

,temp_index,listing_id,id,date,reviewer_id,reviewer_name,comments,cleaned_text
0,0,6422,1927,2009-04-30,14100,Melissa,I can't say enough about how wonderful it was ...,i cant say enough about how wonderful it was t...
1,1,6422,3867,2009-06-11,17413,Raquel,Michelle and Collier's home is wonderful! They...,michelle and colliers home is wonderful they a...
2,2,6422,4159,2009-06-17,20253,Ulrike,I spent one night at Michele's home and felt j...,i spent one night at micheles home and felt ju...
3,3,6422,5724,2009-07-18,22544,Phil,Michele and Collier are two of the loveliest p...,michele and collier are two of the loveliest p...
4,4,6422,11891,2009-09-29,33409,Claire,We had the most lovely time staying with Miche...,we had the most lovely time staying with miche...


In [8]:
#df_reviews['tokens'] = df_reviews['cleaned_text'][2000000:3000000].apply(tokenize_and_lemmatize)

In [9]:
progress_bar = IntProgress(min=0,max=len(df_reviews['cleaned_text'][8000000:]), value=0)

label = HTML(value='Progress: 0%')

display(VBox([label, progress_bar]))

tokens_list = []

for i in df_reviews['cleaned_text'][8000000:]:
    token = tokenize_and_lemmatize(i)
    tokens_list.append(token)
    progress_bar.value = len(tokens_list)
    label.value = f'Progress: {len(tokens_list)/len(df_reviews["cleaned_text"][8000000:]) * 100}%'

#df_reviews['tokens'] = tokens_list

In [10]:
df_reviews2 = df_reviews[8000000:]
df_reviews2['tokens'] = tokens_list
#df_reviews['tokens'] = tokens_list

C:\Users\asl4a\AppData\Local\Temp\ipykernel_1204\3837313392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews2['tokens'] = tokens_list


In [11]:
df_reviews2

,temp_index,listing_id,id,date,reviewer_id,reviewer_name,comments,cleaned_text,tokens
8000000,9772,2898991,36504216,2015-06-29,27101081,Liz,This house was extremely clean and well kept -...,this house was extremely clean and well kept ...,"[house, extremely, clean, keep, short, walk, d..."
8000001,9773,2898991,37278315,2015-07-06,2429321,Maria,"Great location in Newport, walking distance to...",great location in newport walking distance to ...,"[great, location, newport, walk, distance, tow..."
8000002,9774,2898991,37885665,2015-07-11,22445124,James,Let first start by saying how responsive Georg...,let first start by saying how responsive georg...,"[let, start, say, responsive, george, moment, ..."
8000003,9775,2898991,38094545,2015-07-13,6990312,Kara,George was very helpful and accessible through...,george was very helpful and accessible through...,"[george, helpful, accessible, weekend, sleep, ..."
8000004,9776,2898991,38977268,2015-07-20,2206119,Andrew,The place is great. Excellent location and per...,the place is great excellent location and perf...,"[place, great, excellent, location, perfect, g..."
...,...,...,...,...,...,...,...,...,...
9668040,1459400,904459169206385872,906632098517736492,2023-06-04,233078624,Chien-Hao,A very smooth stay in this place. The host is ...,a very smooth stay in this place the host is v...,"[smooth, stay, place, host, responsive, room, ..."
9668041,1459401,902262639204577592,905911996152427208,2023-06-03,93526779,Celeine,The unit was new and very clean. It was a litt...,the unit was new and very clean it was a littl...,"[unit, new, clean, little, tricky, find, find,..."
9668042,1459402,902275771323672573,904510451196355984,2023-06-01,517637293,Maria,"Everything was very easy, quick to book, and t...",everything was very easy quick to book and the...,"[easy, quick, book, place, clean, exactly, pic..."
9668043,1459403,902279125007633591,906577547506923550,2023-06-04,99532747,Andres,This was a great space for two people! Its sma...,this was a great space for two people its smal...,"[great, space, people, small, effective, night..."


In [13]:
df_reviews2.to_csv('Cleaned_Reviews_9.csv')

In [19]:
len(df_reviews['tokens'])

KeyError: 'tokens'

In [ ]:
df_reviews = df_reviews.dropna(axis=0)

In [ ]:
len(df_reviews)

In [ ]:
df_reviews.to_csv('Cleaned_Reviews_4.csv')

In [ ]:
#df_reviews.to_csv('Spacy_Processed_Reviews.csv',index=True)

In [ ]:
# pl.scan_csv() returns a LazyFrame but pl.read_csv() returns a regular polars dataframe
df_reviews = pl.scan_csv(r"C:\Users\asl4a\df_reviews.csv")

In [ ]:
df_reviews.head().collect()

In [ ]:
#df_reviews['listing_id'].value_counts(sort=True)

In [ ]:
df_revs2 = pl.read_csv(r"C:\Users\asl4a\df_reviews.csv")
df_revs2.head()

In [ ]:
df_revs2['listing_id'].value_counts(sort=True)

In [ ]:
df_revs2['listing_id'].n_unique()

In [ ]:
df_revs2['cleaned_text'] = df_revs2.with_columns(['comments'].map_rows(clean_text))

In [ ]:
df_listings = pd.read_sql("""
SELECT *
FROM listings""",con=conn)

In [ ]:
df_listings.head()

In [ ]:
df_listings['host_response_rate'] = df_listings['host_response_rate'].str.strip('%')
df_listings['host_acceptance_rate'] = df_listings['host_acceptance_rate'].str.strip('%')
df_listings['price'] = df_listings['price'].str.strip('$')

In [ ]:
sns.histplot(df_listings['price'])